In [1]:
#import packages
from scipy.stats import norm, t
import scipy
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import sys 
import os
sys.path.append(os.path.abspath("/Users/pudong/Desktop"))
import risklib
from datetime import datetime
from functools import partial
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')
from __future__ import division
from numpy.linalg import inv,pinv
from scipy.optimize import minimize
import copy

In [2]:
# Question 1(a)

In [229]:
#Read data
q1 = pd.read_csv('finalproblem1.csv', index_col=3)

In [71]:
#Calculate Log Returns
def pd_return_calculate(prices, method="arithmetic"):
    price_change_percent = (prices / prices.shift(1))[1:]
    if method == "arithmetic":
        return price_change_percent - 1
    elif method == "log":
        return np.log(price_change_percent)

In [74]:
log_return = pd_return_calculate(q1, method="log")
print('The log returns are as the following:')
print(log_return)

The log returns are as the following:
              Price1    Price2    Price3
Date                                    
2023-04-13  0.014389 -0.001276  0.012089
2023-04-14  0.000350  0.018212 -0.018281
2023-04-15       NaN -0.027514 -0.004008
2023-04-16       NaN  0.015923       NaN
2023-04-17  0.019789  0.041891       NaN
2023-04-18  0.011155 -0.008922 -0.001463
2023-04-19 -0.016460 -0.053688 -0.012862
2023-04-20 -0.001548  0.039996  0.030623
2023-04-21 -0.003592       NaN       NaN
2023-04-22  0.004768       NaN       NaN
2023-04-23       NaN       NaN -0.033571
2023-04-24       NaN       NaN  0.017242
2023-04-25  0.010076  0.004917  0.025432
2023-04-26  0.001086 -0.007237 -0.023922
2023-04-27 -0.006507 -0.023310  0.028445
2023-04-28 -0.005505  0.053597 -0.027276
2023-04-29  0.002584 -0.053102  0.016422
2023-04-30  0.005423  0.035326  0.004032
2023-05-01 -0.015111 -0.022887 -0.021360


In [63]:
# Question 1(b)
correlation_matrix = q1.corr()
correlation_matrix
print('Pairwise correlation matrix:\n')
print(correlation_matrix)

Pairwise correlation matrix:

          Price1    Price2    Price3
Price1  1.000000  0.293073 -0.206321
Price2  0.293073  1.000000 -0.049936
Price3 -0.206321 -0.049936  1.000000


In [66]:
# Question 1(c)
def is_psd(matrix, tol=1e-7):
    return np.all(np.linalg.eigvals(matrix) >= -tol)

#print comfirm result
print(is_psd(correlation_matrix))
print('This matrix is PSD from the test above.')

True
This matrix is PSD from the test above.


In [228]:
# Question 1(d)

Missing values are very common in finance. Not all markets are open at the same time on the same days.
A holiday in one market is not necessarily a holiday in another, even in the same country.
If we only use the days on which all markets are open, the data is very limited and we will lose a 
large portion of the data. Finding the matching rows for each pair is a good way, although we do not
garentee that the matrix is PSD.

In [39]:
#Question 2(a)

In [41]:
def gbsm(option_type, S, X, T, sigma, r, b):
    '''
    option_type: call or put
    S: Underlying price
    X: Strike price
    T: Time to maturity
    sigma: Implied volatility
    r: Risk free rate
    b: Cost of carry
    '''
    d1 = (np.log(S / X) + (b + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    is_call = 1 if option_type == "Call" else -1

    res = is_call * (S * np.e ** ((b - r) * T) * \
                   scipy.stats.norm(0, 1).cdf(is_call * d1) \
                   - X * np.e ** (-r * T) * \
                   scipy.stats.norm(0, 1).cdf(is_call * d2))
    
    return res

In [231]:
S=85.65844188147227
X = 84.76917628074074
T = 142/365
sigma = 0.23
r = 0.045
coupon = 0.051785406106137744
b = r - coupon

In [48]:
# call price
call_price = gbsm('Call', S, X, T, sigma, r, b)
print('Call price is:', call_price)

Call price is: 5.114843950375857


In [49]:
# Question 2(b)

In [232]:
def calculate_d1(S, X, T, sigma, b):
    return (np.log(S / X) + (b + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))

def calculate_d2(d1, T, sigma):
    return d1 - sigma * np.sqrt(T)

In [233]:
def gbsm_delta(option_type, S, X, T, sigma, r, b):
    is_call = 1 if option_type == "Call" else -1
    d1 = calculate_d1(S, X, T, sigma, b)
    delta = norm.cdf(d1 * is_call, 0, 1) * is_call
    return delta

def gbsm_gamma(option_type, S, X, T, sigma, r, b):
    d1 = calculate_d1(S, X, T, sigma, b)
    d2 = calculate_d2(d1, T, sigma)
    gamma = norm.pdf(d1, 0, 1) / (S * sigma * np.sqrt(T))
    return gamma

def gbsm_vega(option_type, S, X, T, sigma, r, b):
    d1 = calculate_d1(S, X, T, sigma, b)
    d2 = calculate_d2(d1, T, sigma)
    vega = S * norm.pdf(d1, 0, 1) * np.sqrt(T)
    return vega

def gbsm_theta(option_type, S, X, T, sigma, r, b):
    is_call = 1 if option_type == "Call" else -1
    d1 = calculate_d1(S, X, T, sigma, b)
    d2 = calculate_d2(d1, T, sigma)
    theta = -S * np.exp((b - r) * T) * norm.pdf(d1, 0, 1) * sigma / (2 * np.sqrt(T)) \
          -(b - r) * S * np.exp((b - r) * T) * norm.cdf(d1 * is_call, 0, 1) * is_call \
          -r * X * np.exp(-r * T) * norm.cdf(d2 * is_call, 0, 1) * is_call
    return theta

def gbsm_rho(option_type, S, X, T, sigma, r, b):
    is_call = 1 if option_type == "Call" else -1
    d1 = calculate_d1(S, X, T, sigma, b)
    d2 = calculate_d2(d1, T, sigma)
    rho = X * T * np.exp(-r * T) * norm.cdf(d2 * is_call, 0, 1) * is_call
    return rho

def gbsm_carry_rho(option_type, S, X, T, sigma, r, b):
    is_call = 1 if option_type == "Call" else -1
    d1 = calculate_d1(S, X, T, sigma, b)
    d2 = calculate_d2(d1, T, sigma)
    carry_rho = S * T * np.exp((b - r) * T) * norm.cdf(d1 * is_call, 0, 1) * is_call
    return carry_rho

In [234]:
# Question 2(b)
#Delta
delta = gbsm_delta('Call', S, X, T, sigma, r, b)
print('Delta is:', delta)

Delta is: 0.5501626934623628


In [161]:
#Question 2(c)
#Gamma
gamma = gbsm_gamma('Call', S, X, T, sigma, r, b)
print('Gamma is:', gamma)

Gamma is: 0.03220793839235446


In [56]:
#Question 2(d)
#Vega
vega = gbsm_vega('Call', S, X, T, sigma, r, b)
print('Vega is:', vega)

Vega is: 21.14591993424314


In [57]:
#Question 2(e)
#Rho
rho = gbsm_rho('Call', S, X, T, sigma, r, b)
print('Rho is:', rho)

Rho is: 15.978424540858985


In [ ]:
#Question 2(f)


In [75]:
# Question 3(a)

In [78]:
# Read data
q3cov = pd.read_csv('problem3_cov.csv')
q3er = pd.read_csv('problem3_ER.csv')

In [85]:
stocks = ['A','B','C']
cov_matrix = np.matrix(q3cov)
expect_return = np.matrix(q3er.drop(['RF'], axis=1))

In [88]:
def neg_sharpe_ratio(weights, mean, cov, r):
    returns = mean @ weights.T
    std = np.sqrt(weights @ cov @ weights.T)
    return -(returns - r) / std

args = (expect_return, cov_matrix, 0.045)
bounds = [(0.0, 1) for _ in stocks]
x0 = np.array(3*[1 / 3])
constraints = {'type':'eq', 'fun': lambda x: np.sum(x) - 1}
results = scipy.optimize.minimize(neg_sharpe_ratio, x0=x0, args=args, bounds = bounds, constraints=constraints)
opt_sharpe, opt_weights = -results.fun, pd.Series(results.x, index=stocks)
opt_weights = pd.DataFrame(opt_weights, columns=['weights(%)'])
opt_weights['weights(%)'] = round(opt_weights*100, 2)

print("The most efficient portfolio consists of: ")
display(opt_weights)
print("The Portfolio's Sharpe Ratio is: " + str(opt_sharpe))

The most efficient portfolio consists of: 


,weights(%)
A,34.55
B,34.87
C,30.59


The Portfolio's Sharpe Ratio is: 0.5566581406854578


In [94]:
# Question 3(b)
 # risk budgeting optimization
def calculate_portfolio_var(w,V):
    # function that calculates portfolio risk
    w = np.matrix(w)
    return (w*V*w.T)[0,0]

def calculate_risk_contribution(w,V):
    # function that calculates asset contribution to total risk
    w = np.matrix(w)
    sigma = np.sqrt(calculate_portfolio_var(w,V))
    # Marginal Risk Contribution
    MRC = V*w.T
    # Risk Contribution
    RC = np.multiply(MRC,w.T)/sigma
    return RC

def risk_budget_objective(x,pars):
    # calculate portfolio risk
    V = pars[0]# covariance table
    x_t = pars[1] # risk target in percent of portfolio risk
    sig_p =  np.sqrt(calculate_portfolio_var(x,V)) # portfolio sigma
    risk_target = np.asmatrix(np.multiply(sig_p,x_t))
    asset_RC = calculate_risk_contribution(x,V)
    J = sum(np.square(asset_RC-risk_target.T))[0,0] # sum of squared error
    return J

def total_weight_constraint(x):
    return np.sum(x)-1.0

def long_only_constraint(x):
    return x

x_t = np.array([1, 1, 1]) / 3 # your risk budget percent of total portfolio risk (equal risk)
w0 = np.array([1, 1, 1]) / 3 
cons = ({'type': 'eq', 'fun': total_weight_constraint},
{'type': 'ineq', 'fun': long_only_constraint})
res= minimize(risk_budget_objective, w0, args=[cov_matrix,x_t], method='SLSQP',constraints=cons, options={'disp': True})
w_rb = np.asmatrix(res.x)
print(w_rb)
print('The above matrix is the corresponding weight for risk parity portfolio.')

Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.009876629908044e-08
            Iterations: 4
            Function evaluations: 16
            Gradient evaluations: 4
[[0.34576234 0.34806117 0.30617649]]
The above matrix is the corresponding weight for risk parity portfolio.


In [131]:
# Question 3(c)
opt_w = np.matrix(opt_weights)/100
RP_w = w_rb
ER_O = expect_return*opt_w
ER_R = expect_return*RP_w.T
SD_O = np.sqrt(opt_w.T*cov_matrix*opt_w)
SD_R = np.sqrt(RP_w*cov_matrix*RP_w.T)
SR_O = (expect_return*opt_w-0.045)/SD_O 
SR_R = (expect_return*RP_w.T-0.045)/SD_R
print('ER optimal:', ER_O)
print('ER RP:', ER_R)
print('SD optimal:', SD_O)
print('SD RP:', SD_R)
print('SR optimal:', SR_O)
print('SR RP:', SR_R)

ER optimal: [[0.12098371]]
ER RP: [[0.12097494]]
SD optimal: [[0.1364917]]
SD RP: [[0.1364841]]
SR optimal: [[0.55669111]]
SR RP: [[0.55665782]]


In [235]:
# Question 3(c)

The weights are very similar to each other, and it's almost identical with just a little difference.
We can see that from the above ER, SD, and SR analysis that these three numbers for optimal portfolio and
for risk parity portfolio are very similar and very close to each other. This can also explain why
the weights for two portfolios are very close to each other.

In [225]:
# Question 4(b) & 4(c) Note that 4(a) is followed by (b)&(c)

In [162]:
q4 = pd.read_csv("problem4_returns.csv").drop(['Date'], axis=1)
port_weight = np.matrix(pd.read_csv('problem4_startWeight.csv'))
portfolio_weight=[0.3457527 , 0.27249649, 0.38175081]

In [205]:
# Get Updated Prices
stocks=['Asset1', 'Asset2', 'Asset3']
portfolio_weight=[0.3457527 , 0.27249649, 0.38175081]

# Calculate portfolio return and updated weights for each day
n = q4.shape[0]
m = len(stocks)

pReturn = np.empty(n)
weights = np.empty((n, len(portfolio_weight)))
lastW = portfolio_weight.copy()
matReturns = q4[stocks].values
update_weights = []
for i in range(n):
    # Save Current Weights in Matrix
    weights[i, :] = lastW

    # Update Weights by return
    lastW = lastW * (1.0 + matReturns[i, :])

    # Portfolio return is the sum of the updated weights
    pR = lastW.sum()

    # Normalize the weights back so sum = 1
    lastW = lastW / pR
    update_weights.append(lastW)

    # Store the return
    pReturn[i] = pR - 1

# Set the portfolio return in the Update Return DataFrame
q4["Portfolio"] = pReturn
# Calculate the total return
totalRet = np.exp(np.sum(np.log(pReturn + 1))) - 1
# Calculate the Carino K
k = np.log(totalRet + 1) / totalRet

# Carino k_t is the ratio scaled by 1/K
carinoK = np.log(1.0 + pReturn) / pReturn / k

# Calculate the return attribution
attrib = pd.DataFrame(matReturns * weights * carinoK[:, np.newaxis], columns=stocks)

# Set up a DataFrame for output
Attribution = pd.DataFrame({"Value": ["TotalReturn", "Return Attribution"]})

# Loop over the stocks
for s in stocks + ["Portfolio"]:
    # Total Stock return over the period
    tr = np.exp(np.sum(np.log(q4[s] + 1))) - 1

    # Attribution Return (total portfolio return if we are updating the portfolio column)
    atr = tr if s == "Portfolio" else attrib[s].sum()

    # Set the values
    Attribution[s] = [tr, atr]

# Realized Volatility Attribution

# Y is our stock returns scaled by their weight at each time
Y = matReturns * weights

# Set up X with the Portfolio Return
X = np.column_stack((np.ones(n), pReturn))

# Calculate the Beta and discard the intercept
B = np.linalg.inv(X.T @ X) @ X.T @ Y
B = B[1, :]

# Component SD is Beta times the standard Deviation of the portfolio
cSD = B * np.std(pReturn)

# Add the Vol attribution to the output
vol_attrib = pd.DataFrame({"Value": ["Vol Attribution"], **{stocks[i]: [cSD[i]] for i in range(len(stocks))}, "Portfolio": [np.std(pReturn)]})

Attribution = pd.concat([Attribution, vol_attrib], ignore_index=True)
print('Return Attribution and risk Attribution for each asset are in the following table:')
print(Attribution)

Return Attribution and risk Attribution for each asset are in the following table:
                Value    Asset1    Asset2    Asset3  Portfolio
0         TotalReturn -0.281192  0.356570  0.581352   0.221873
1  Return Attribution -0.086871  0.087059  0.221685   0.221873
2     Vol Attribution  0.009658  0.004309  0.016932   0.030898


In [224]:
# Question 4(a)
new_weights = pd.DataFrame(update_weights)
q4date = pd.read_csv("problem4_returns.csv")['Date']
new_weights['Date'] = q4date
print('The new weights for each time period is as the following:')
print(new_weights)

The new weights for each time period is as the following:
           0         1         2        Date
0   0.340701  0.276936  0.382363  2023-04-12
1   0.342880  0.264561  0.392559  2023-04-13
2   0.345686  0.245843  0.408471  2023-04-14
3   0.345938  0.256509  0.397553  2023-04-15
4   0.336285  0.248698  0.415018  2023-04-16
5   0.327108  0.247798  0.425094  2023-04-17
6   0.315249  0.245591  0.439161  2023-04-18
7   0.313195  0.224855  0.461951  2023-04-19
8   0.302968  0.236569  0.460463  2023-04-20
9   0.289797  0.247324  0.462879  2023-04-21
10  0.299361  0.239763  0.460876  2023-04-22
11  0.279797  0.225325  0.494878  2023-04-23
12  0.281712  0.231770  0.486518  2023-04-24
13  0.277160  0.255895  0.466945  2023-04-25
14  0.265720  0.255861  0.478419  2023-04-26
15  0.260515  0.264125  0.475360  2023-04-27
16  0.245631  0.278970  0.475399  2023-04-28
17  0.220813  0.302848  0.476339  2023-04-29
18  0.211770  0.303685  0.484546  2023-04-30
19  0.203401  0.302536  0.494063  2023-05-

In [196]:
# Question 5
prices = pd.read_csv("problem5.csv").drop(['Date'],axis=1)
returns = risklib.pd_calculate_returns(prices)
portfolios = pd.DataFrame()
portfolios['Portfolio'] = ['Asset1', 'Asset2', 'Asset3', 'Asset4', 'Asset1&2', 'Asset1&2', 'Asset3&4', 'Asset3&4', 'Total','Total','Total','Total']
portfolios['Stock'] = ['Price1', 'Price2', 'Price3', 'Price4', 'Price1', 'Price2', 'Price3', 'Price4', 'Price1','Price2','Price3','Price4']
portfolios['Holding'] = [1,1,1,1,1,1,1,1,1,1,1,1]

In [197]:
# Calculate t VaR.
current_prices = pd.DataFrame({"Price":prices.iloc[-1]})

for portfolio_index, portfolio in portfolios.groupby('Portfolio'):
    portfolio = portfolio.set_index('Stock')
    portfolio = portfolio.join(current_prices.loc[portfolio.index])

    current_values = portfolio['Holding'] * portfolio['Price']
    portfolio_value = current_values.sum()

    sim_returns = returns[portfolio.index]
    sim_prices = (1 + sim_returns) * portfolio['Price'].T
    sim_values = sim_prices @ portfolio['Holding']

    t_var = risklib.t_var(sim_values, portfolio_value)
    print(f"Portfolio {portfolio_index}: " + str(t_var))

Portfolio Asset1: 0.04060626910421661
Portfolio Asset1&2: 0.08256019979373264
Portfolio Asset2: 0.060685296702743585
Portfolio Asset3: 0.0538184428363877
Portfolio Asset3&4: 0.08618808770157216
Portfolio Asset4: 0.044899987915400175
Portfolio Total: 0.13797841918773202


In [201]:
# Question 5(a)
print('Portfolio Asset1 VaR 5%: 0.04060626910421661')
print('Portfolio Asset2 VaR 5%: 0.060685296702743585')
print('Portfolio Asset3 VaR 5%: 0.0538184428363877')
print('Portfolio Asset4 VaR 5%: 0.044899987915400175')

Portfolio Asset1 VaR 5%: 0.04060626910421661
Portfolio Asset2 VaR 5%: 0.060685296702743585
Portfolio Asset3 VaR 5%: 0.0538184428363877
Portfolio Asset4 VaR 5%: 0.044899987915400175


In [203]:
# Question 5(b)
print('Portfolio Asset1&2 VaR 5%: 0.08256019979373264')
print('Portfolio Asset3&4 VaR 5%: 0.08618808770157216')

Portfolio Asset1&2 VaR 5%: 0.08256019979373264
Portfolio Asset3&4 VaR 5%: 0.08618808770157216


In [204]:
# Question 5(c)
print('Portfolio Total all 4 assets VaR 5%: 0.13797841918773202')

Portfolio Total all 4 assets VaR 5%: 0.13797841918773202
